# Retrieve NGA WPI 

import pandas as pd
import requests
import geopandas
import seaconex
import json

from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file
Retrieve MSSIS ports from [Pub. 150, The World Port Index, National Geospatial-Intelligence Agency (NGA), Maritime Safety Office](https://msi.nga.mil/Publications/WPI) 

- [NGA Maritime Safety Information REST API Documentation](https://msi.nga.mil/api/swagger-ui.html)



In [ ]:
import pandas as pd
import requests
import geopandas
import seaconex
import json
import numpy as np

from shapely.geometry import Point
from geopandas import GeoDataFrame, read_file

In [ ]:
mssis_ais_region = '../data/interim/mssis-ais-region_v2.json'
mssis_port_visit_ports = '../data/interim/mssis-port-visit-ports_v1.json'
mssis_wpi_bm_ports = '../data/interim/mssis-wpi-bm-ports_v1.json'

In [ ]:
df_ais = pd.read_json(mssis_ais_region)

In [ ]:
df_port = pd.read_json(mssis_port_visit_ports)

In [ ]:
df_bm = pd.read_json(mssis_wpi_bm_ports)

In [ ]:
df_ais.head()

In [ ]:
df_port.head()

In [ ]:
df_bm.head()

In [ ]:
df_bm = df_bm.filter(like='wpi').drop(columns=['mssis_wpi_region_index'])

In [ ]:
# result = pd.concat([df_ais, df_port, df_bm], ignore_index=True, sort=True)

In [ ]:
# result.head()

In [ ]:
# result.filter(like='mssis').head()

In [ ]:
# df_port.append(df_bm, ignore_index=True)

In [ ]:
# merged_inner = pd.merge(left=df_port, right=df_bm, left_on='mssis_wpi', right_on='mssis_wpi')

In [ ]:
# merged_inner

In [ ]:
# join_col = df_port.columns & df_bm.columns

In [ ]:
join_col = np.intersect1d(df_port.columns, df_bm.columns).tolist()

In [ ]:
join_col

In [ ]:
df = pd.merge(left=df_port, right=df_bm, how='outer', left_on=join_col, right_on=join_col)

In [ ]:
join_col = np.intersect1d(df.columns, df_ais.columns).tolist()

In [ ]:
join_col

In [ ]:
result = pd.merge(left=df, right=df_ais, how='outer', left_on=join_col, right_on=join_col)

In [ ]:
# result

In [ ]:
result.info()

In [ ]:
result.head()

In [ ]:
result[['ais_count']] = result[['ais_count']].fillna(0).astype('int')

In [ ]:
result

In [ ]:
# result[['mssis_wpi_region_index', 'mssis_ao', 'ais_count']] = result[['mssis_wpi_region_index', 'mssis_ao', 'ais_count']].fillna(0).astype('int')

In [ ]:
# result

In [ ]:
result.sort_values('mssis_wpi').sort_values('ais_count')

In [ ]:
wpi_list = result[result['mssis_wpi'] > 0].sort_values('mssis_wpi').mssis_wpi.unique().tolist()

In [ ]:
wpi_list

In [ ]:
nga_wpi = seaconex.nga_msi_wpi_api_get_world_port_index(wpi_list)

In [ ]:
nga = pd.DataFrame(nga_wpi['ports'])

In [ ]:
nga.to_json('../data/raw/' + 'nga-bermuda-wpi_v1.json')

In [ ]:
nga.to_json('../data/raw/' + 'nga-bermuda-wpi_records_v1.json', orient='records')

In [ ]:
nga.to_json('../data/raw/' + 'nga-bermuda-wpi_split_v1.json', orient='split')

In [ ]:
nga.to_json('../data/raw/' + 'nga-bermuda-wpi_table_v1.json', orient='table')

In [ ]:
# nga_wpi_prc = seaconex.nga_msi_wpi_api_get_prc_names()

In [ ]:
# nga_wpi_prc

In [ ]:
nga.head()

In [ ]:
geometry = geopandas.points_from_xy(nga.xcoord, nga.ycoord)

In [ ]:
nga.drop(columns=['ycoord', 'xcoord', 'latitude', 'longitude'], inplace=True)

In [ ]:
null_columns=nga.columns[nga.isnull().all()]

In [ ]:
null_columns

In [ ]:
null_columns.tolist()

In [ ]:
nga.drop(columns=null_columns, inplace=True)

In [ ]:
print(nga.columns.tolist())

In [ ]:
nga.columns.tolist()

In [ ]:
pd.set_option("max_columns", None)
pd.set_option("max_rows", None)

In [ ]:
drop_col = ['shelter', 'erTide', 'erSwell', 'erIce', 'erOther', 'overheadLimits', 'chDepth','anDepth','cpDepth','otDepth','tide', 'goodHoldingGround', 'turningArea', 'firstPortOfEntry', 'usRep', 'ptCompulsory', 'ptAdvisable', 'tugsSalvage', 'tugsAssist', 'qtPratique', 'qtOther', 'cmTelephone', 'cmTelegraph', 'cmRadio', 'cmRadioTel', 'cmAir', 'cmRail', 'loWharves', 'loAnchor', 'loMedMoor', 'loBeachMoor', 'loIceMoor', 'medFacilities', 'garbageDisposal', 'degauss', 'dirtyBallast', 'crFixed', 'crMobile', 'crFloating', 'lifts100', 'lifts50', 'lifts25', 'lifts0', 'srLongshore', 'srElectrical', 'srSteam', 'srNavigEquip', 'srElectRepair', 'suProvisions', 'suWater', 'suFuel', 'suDiesel', 'suDeck', 'repairCode', 'drydock', 'railway', 'qtSanitation', 'suAviationFuel', 'harborUse', 'ukcMgmtSystem', 'portSecurity', 'etaMessage', 'searchAndRescue', 'tss', 'vts', 'cht', 'loRoro', 'loSolidBulk', 'loContainer', 'loBreakBulk', 'loOilTerm', 'loLongTerm', 'loOther', 'loDangCargo', 'loLiquidBulk', 'srIceBreaking', 'srDiving', 'cranesContainer', 's121WaterBody', 's101Enc', 'dodWaterBody']

In [ ]:
nga.drop(columns=drop_col, inplace=True)

In [ ]:
nga

In [ ]:
gdf = geopandas.GeoDataFrame(nga, geometry=geometry, crs="EPSG:4326")

In [ ]:
gdf.head()

In [ ]:
gdf.crs

In [ ]:
gdf.to_file('../data/interim/' + 'nga-wpi_v1.json', driver='GeoJSON')

In [ ]:
gdf.to_file('../data/interim/' + 'nga-wpi_v1.gpkg', driver='GPKG')

In [ ]:
# are these ports missing??
salem = '8025'
cork = '34360'
london = '31470'